Generators of Groups

In [ ]:
def power_mod(base, exp, mod):
    result = 1
    while exp > 0:
        if exp % 2 == 1:
            result = (result * base) % mod
        base = (base * base) % mod
        exp //= 2
    return result

def is_primitive_root(g, p, factors):
    for q in factors:
        if power_mod(g, (p-1) // q, p) == 1:
            return False
    return True

p = 28151
p_minus_1 = p - 1
factors = [2, 5, 25, 563]
g = 2
while not is_primitive_root(g, p, factors):
    g += 1

print(g)

Computing Public Values

In [ ]:
g = 2
p = 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919
a = 972107443837033796245864316200458246846904598488981605856765890478853088246897345487328491037710219222038930943365848626194109830309179393018216763327572120124760140018038673999837643377590434413866611132403979547150659053897355593394492586978400044375465657296027592948349589216415363722668361328689588996541370097559090335137676411595949335857341797148926151694299575970292809805314431447043469447485957669949989090202320234337890323293401862304986599884732815

result = pow(g, a, p)

print(result)

Computing Shared Secrets

In [ ]:
p = 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919
A = 70249943217595468278554541264975482909289174351516133994495821400710625291840101960595720462672604202133493023241393916394629829526272643847352371534839862030410331485087487331809285533195024369287293217083414424096866925845838641840923193480821332056735592483730921055532222505605661664236182285229504265881752580410194731633895345823963910901731715743835775619780738974844840425579683385344491015955892106904647602049559477279345982530488299847663103078045601
b = 12019233252903990344598522535774963020395770409445296724034378433497976840167805970589960962221948290951873387728102115996831454482299243226839490999713763440412177965861508773420532266484619126710566414914227560103715336696193210379850575047730388378348266180934946139100479831339835896583443691529372703954589071507717917136906770122077739814262298488662138085608736103418601750861698417340264213867753834679359191427098195887112064503104510489610448294420720

shared_secret = pow(A, b, p)
print(shared_secret)

Deriving Symmetric Keys

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
import hashlib

def is_pkcs7_padded(message):
    padding = message[-message[-1]:]
    return all(p == len(padding) for p in padding)

def decrypt_flag(shared_secret: int, iv: str, ciphertext: str):
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]

    ciphertext_bytes = bytes.fromhex(ciphertext)
    iv_bytes = bytes.fromhex(iv)
    cipher = AES.new(key, AES.MODE_CBC, iv_bytes)
    plaintext = cipher.decrypt(ciphertext_bytes)

    if is_pkcs7_padded(plaintext):
        return unpad(plaintext, 16).decode('ascii')
    else:
        return plaintext.decode('ascii')

g = 2
p = 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919
A = 112218739139542908880564359534373424013016249772931962692237907571990334483528877513809272625610512061159061737608547288558662879685086684299624481742865016924065000555267977830144740364467977206555914781236397216033805882207640219686011643468275165718132888489024688846101943642459655423609111976363316080620471928236879737944217503462265615774774318986375878440978819238346077908864116156831874695817477772477121232820827728424890845769152726027520772901423784
b = 197395083814907028991785772714920885908249341925650951555219049411298436217190605190824934787336279228785809783531814507661385111220639329358048196339626065676869119737979175531770768861808581110311903548567424039264485661330995221907803300824165469977099494284722831845653985392791480264712091293580274947132480402319812110462641143884577706335859190668240694680261160210609506891842793868297672619625924001403035676872189455767944077542198064499486164431451944
shared_secret = pow(A, b, p)
iv = '737561146ff8194f45290f5766ed6aba'
ciphertext = '39c99bf2f0c14678d6a5416faef954b5893c316fc3c48622ba1fd6a9fe85f3dc72a29c394cf4bc8aff6a7b21cae8e12c'
flag = decrypt_flag(shared_secret, iv, ciphertext)
print(flag)

Parameter Injection

In [ ]:
import socket
import json
import hashlib
from Crypto.Cipher import AES

def enc(s):
    s.connect((HOST,PORT))
    s.recv(1024)
    data = s.recv(804)
    data2 = json.loads(data)
    p = int(data2['p'],16)
    A = int(data2['A'],16)
    s.recv(100)
    s.send(data)
    s.recv(1024)
    data3 = s.recv(404-9)
    data3 = b'{"B" : "0x83e907190b6484aa982847f873111a28a3f1a0617a0973b24f8ed036d01d01009f050fa636cfe030cdd26f1309465cdea4ebc97d421fa5ebeedda63d948c8b00e81c8e8e63e720ad74bf867139ac2112883928d0441290f9f40e67a44e4447b7f8841f6f573b8b6a85d679bb611d7f026a4c2c904dd4a97a2d0048531f43b78e7c539d9e59149229ed32630506d11f13b42609bb4b8c4644e0f3ede537022ac7de96288c1794746f3f57b25a2668363a4314879c3834a9961ba3800f7de4798d"}'

    s.recv(100)
    s.send(data3)
    s.recv(20+4)
    enc = s.recv(1000).strip().decode()
    enc = json.loads(enc)
    iv = enc['iv']
    enc_flag = enc['encrypted_flag']
    return A,p,enc_flag,iv

def decrypt(shared_key,enc_flag,iv):
    sha1 = hashlib.sha1()
    sha1.update(str(shared_key).encode('ascii'))
    key = sha1.digest()[:16]
    ciphertext = bytes.fromhex(enc_flag)
    iv = bytes.fromhex(iv)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext)
    return plaintext

if __name__ == "__main__":
    HOST = "socket.cryptohack.org"
    PORT = 13371

    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    Alice_pub,prime,enc_flag,iv = enc(sock)
    private_key = 197395083814907028991785772714920885908249341925650951555219049411298436217190605190824934787336279228785809783531814507661385111220639329358048196339626065676869119737979175531770768861808581110311903548567424039264485661330995221907803300824165469977099494284722831845653985392791480264712091293580274947132480402319812110462641143884577706335859190668240694680261160210609506891842793868297672619625924001403035676872189455767944077542198064499486164431451944
    shared_key = pow(Alice_pub,private_key,prime)
    flag = decrypt(shared_key,enc_flag,iv).decode()
    print(flag)

Export Grade

In [ ]:
from Crypto.Util.number import long_to_bytes
from Crypto.Cipher import AES
from math import gcd

from socket import socket
import hashlib
import random
import json

def is_pkcs7_padded(message):
    padding = message[-message[-1]:]
    return all(padding[i] == len(padding) for i in range(0, len(padding)))

buf = ""
def readline(r):
    buf = ""
    while True:
        data = r.recv(1024).decode()
        buf += data
        if "\n" in data:
            return buf.strip()


def sendline(r, msg):
    r.send(msg + '\n')


r = socket()
r.connect(("socket.cryptohack.org", 13379))

DHs = json.loads(readline(r)[23:-1])

sendline(r, json.dumps({"supported":[DHs['supported'][-1]]}))

DH = json.loads(readline(r)[35:-1])

sendline(r, json.dumps(DH))
alice = json.loads(readline(r)[38:-1])

p = int(alice['p'], 16)
g = int(alice['g'], 16)
A = int(alice['A'], 16)

bob = json.loads(readline(r)[21:-1])

B = int(bob['B'], 16)

ciphertext = json.loads(readline(r)[23:-1])
R = IntegerModRing(p)
a = discrete_log(R(A), R(g))

assert pow(g, a, p) == A

s = pow(B, a, p)

shared_secret = str(s).encode('ascii')

sha1 = hashlib.sha1()
sha1.update(shared_secret)

KEY = sha1.digest()[:16]

cipher = AES.new(KEY, AES.MODE_CBC, ciphertext['iv'].decode('hex'))
plaintext = cipher.decrypt(ciphertext['encrypted_flag'].decode('hex'))

print(plaintext)

Static Client

In [ ]:
import hashlib
import json
import ptrlib
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
sock = ptrlib.Socket("socket.cryptohack.org", 13373)
hex2int = lambda s: int(s[2:], 16)
parse_ct = lambda msg: tuple(bytes.fromhex(msg[x]) for x in ('iv', 'encrypted'))
intercept_alice = lambda: json.loads(sock.recvlineafter('Intercepted from Alice: '))
intercept_bob = lambda: json.loads(sock.recvlineafter('Intercepted from Bob: '))
recv_bob = lambda: json.loads(sock.recvlineafter('Bob says to you: '))
send_bob = lambda payload: sock.sendlineafter('Bob connects to you, send him some parameters: ', json.dumps(payload))

def decrypt_msg(shared_secret, iv, ct):
    key = hashlib.sha1(str(shared_secret).encode('ascii')).digest()[:16]

    cipher = AES.new(key, AES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct), 16)

    return pt

msg = intercept_alice()
p, g, A = (hex2int(msg[x]) for x in 'pgA')

msg = intercept_bob()
B = hex2int(msg['B'])

msg = intercept_alice()
iv, ct = parse_ct(msg)

send_bob({ 'p': hex(p), 'g': hex(A), 'A': hex(0) })

msg = recv_bob()
B2 = hex2int(msg['B'])

msg = recv_bob()
iv2, ct2 = parse_ct(msg)
pt2 = decrypt_msg(0, iv2, ct2)
print(pt2)
pt = decrypt_msg(B2, iv, ct)
print(pt)

Additive

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Util.number import inverse
import hashlib
from sympy.ntheory.residue_ntheory import discrete_log

def is_valid_pkcs7_padding(data):
    padding_length = data[-1]
    return all(data[i] == padding_length for i in range(-padding_length, 0))

def decrypt(shared_secret_int: int, iv_hex: str, encrypted_message_hex: str):
    sha1_hash = hashlib.sha1()
    sha1_hash.update(str(shared_secret_int).encode('utf-8'))
    aes_key = sha1_hash.digest()[:16]
    iv = bytes.fromhex(iv_hex)
    ciphertext = bytes.fromhex(encrypted_message_hex)
    cipher = AES.new(aes_key, AES.MODE_CBC, iv)
    decrypted_message = cipher.decrypt(ciphertext)
    if is_valid_pkcs7_padding(decrypted_message):
        return unpad(decrypted_message, 16).decode('utf-8')
    else:
        return decrypted_message.decode('utf-8')

p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
g = 0x2
A = 0x190766d4be1c2ec1c00f7838764a46288c28567d5e093e49ef71587272f80bed58f8c47c70a99f4b0eceab6d659ce8d2c8475b10f93fc5123017fb54a875a9b18191af5dac0d857c28953fba4d10b2a1dfcb83bbb0f29bfa10fdb15ce8af2b8d441a00b1cf0f2e0634bb0aa912a9a6107b205b225a52a6d822ad7c31c402ae93a6d2d77b5664cae2703433170fdf1703abf7f30f353752ad9cc98db09f6f1e891e20535081cfe86f5e1415d5f2aca3956f2eb6691901d3e55cfcbe3f2a3a550b
B = 0x92092dfaafe2de9fa585c26f79aa763c1069cb6e37d0f68c655d89fc9e99f17703c9336558b8cd49bc72de084a2e7736644d7c79147ee4bd93c8bffe883a9442a2bbf269a07c9af81b56b5af1fe523d7201acd9636e52e3f0a07ae124e986d94560c044a9a62ae916ec89c4450c2a31a3b6c400d21b00bbb292611bf504d5a22282dc767dca91f0573da8fa35a8deaa1307bf272589af97052d6402b69c34313fa5d7a4c06d6cbca70b0e74979821cf09f5f581a2f3fb86c6f455863caa9f0b0
a = A * inverse(g, p)
shared_secret = (a * B) % p
iv = "317c46f08dd3bbe5138d8b53163402ea"
ciphertext = "636a2cab48228912618557d1355cda5dc104839b1db454b5aa62c397c92dd876a56d13a3c291f4dfbfddfd37b837f5e0"
flag = decrypt(shared_secret, iv, ciphertext)
print(flag)

Static Client 2

Script Kiddie

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Util.number import inverse
import hashlib
from sympy.ntheory.residue_ntheory import discrete_log

def is_valid_pkcs7_padding(data):
    padding_length = data[-1]
    return all(data[i] == padding_length for i in range(-padding_length, 0))

def decrypt(shared_secret_int: int, iv_hex: str, encrypted_message_hex: str):
    sha1_hash = hashlib.sha1()
    sha1_hash.update(str(shared_secret_int).encode('utf-8'))
    aes_key = sha1_hash.digest()[:16]
    iv = bytes.fromhex(iv_hex)
    ciphertext = bytes.fromhex(encrypted_message_hex)
    cipher = AES.new(aes_key, AES.MODE_CBC, iv)
    decrypted_message = cipher.decrypt(ciphertext)
    if is_valid_pkcs7_padding(decrypted_message):
        return unpad(decrypted_message, 16).decode('utf-8')
    else:
        return decrypted_message.decode('utf-8')

p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
g = 0x2
A = 0x190766d4be1c2ec1c00f7838764a46288c28567d5e093e49ef71587272f80bed58f8c47c70a99f4b0eceab6d659ce8d2c8475b10f93fc5123017fb54a875a9b18191af5dac0d857c28953fba4d10b2a1dfcb83bbb0f29bfa10fdb15ce8af2b8d441a00b1cf0f2e0634bb0aa912a9a6107b205b225a52a6d822ad7c31c402ae93a6d2d77b5664cae2703433170fdf1703abf7f30f353752ad9cc98db09f6f1e891e20535081cfe86f5e1415d5f2aca3956f2eb6691901d3e55cfcbe3f2a3a550b
B = 0x92092dfaafe2de9fa585c26f79aa763c1069cb6e37d0f68c655d89fc9e99f17703c9336558b8cd49bc72de084a2e7736644d7c79147ee4bd93c8bffe883a9442a2bbf269a07c9af81b56b5af1fe523d7201acd9636e52e3f0a07ae124e986d94560c044a9a62ae916ec89c4450c2a31a3b6c400d21b00bbb292611bf504d5a22282dc767dca91f0573da8fa35a8deaa1307bf272589af97052d6402b69c34313fa5d7a4c06d6cbca70b0e74979821cf09f5f581a2f3fb86c6f455863caa9f0b0
a = A * inverse(g, p)
shared_secret = (a * B) % p
iv = "317c46f08dd3bbe5138d8b53163402ea"
ciphertext = "636a2cab48228912618557d1355cda5dc104839b1db454b5aa62c397c92dd876a56d13a3c291f4dfbfddfd37b837f5e0"
flag = decrypt(shared_secret, iv, ciphertext)
print(flag)

The Matrix

In [ ]:
P = 2
N = 50
E = 31337

def binary2bytes(s):
    return int(s, 2).to_bytes((len(s) + 7) // 8, byteorder='big')

def read_matrix(m):
    l = []
    for y in range(50):
        for x in m:
            l.append(str(x[y]))
    plaintext = ("".join(l))
    return binary2bytes(plaintext[:2480])

def load_matrix(fname):
    data = open(fname, 'r').read().strip()
    rows = [list(map(int, row)) for row in data.splitlines()]
    return Matrix(GF(P), rows)

c = load_matrix("flag.enc")
c = c ** pow(E, -1, c.multiplicative_order())
print(read_matrix(c))

The Matrix Reloaded

In [ ]:
from Crypto.Util.number import isPrime, long_to_bytes, getPrime
from sympy.ntheory.residue_ntheory import discrete_log
from json import loads, dumps
import socket

host = "socket.cryptohack.org"
port = 13378
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((host, port))

def recv_data():
    return s.recv(1024).decode()

A_data = loads(recv_data().split(":", 1)[1].strip())
B_data = loads(recv_data().split(":", 1)[1].strip())
cipher = loads(recv_data().split(":", 1)[1].strip())

p = int(A_data["p"], 16)
A = int(A_data["A"], 16)

i = 2
smooth_p = 1
while smooth_p < p or not isPrime(smooth_p + 1):
    smooth_p *= i
    i += 1
smooth_p += 1

s.sendall(dumps({
    "g": "0x02",
    "A": hex(A),
    "p": hex(smooth_p)
}).encode() + b'\n')

B = int(recv_data().split()[13][1:-2], 16)
b = discrete_log(smooth_p, B, 2)

shared_secret = pow(A, b, p)
print(f"Shared secret: {shared_secret}")
s.close()